In [28]:
!nvidia-smi

Sun Sep 22 15:53:50 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000001:00:00.0 Off |                    0 |
| N/A   67C    P0            304W /  300W |   76797MiB /  81920MiB |     91%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


+-----------------------------------------------------------------------------------------+


In [29]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [30]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration, ParlerTTSConfig
from transformers import AutoTokenizer, GenerationConfig
import soundfile as sf

In [31]:
checkpoint = 'output_dir_training-v3'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
config = ParlerTTSConfig.from_pretrained(checkpoint)

In [32]:
model = ParlerTTSForConditionalGeneration(config = config)
model.generation_config = GenerationConfig.from_pretrained('parler-tts/parler-tts-mini-v1')

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [33]:
!ls {checkpoint}

checkpoint-8100-epoch-2  config.json		   spiece.model
checkpoint-8200-epoch-2  preprocessor_config.json  tokenizer.json
checkpoint-8300-epoch-2  special_tokens_map.json   tokenizer_config.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
checkpoint = torch.load(f'{checkpoint}/checkpoint-8300-epoch-2/pytorch_model.bin')

/tmp/ipykernel_141/1654779978.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f'{checkpoint}/checkpoint-8300-epoch-2/pytorch_model.bin')


In [35]:
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [12]:
model = model.to(device)

In [13]:
model.device

device(type='cuda', index=0)

In [14]:
prompt = "Husein zolkepli sangat comel dan kacak suka makan cendol"
description = "Ariff's voice is screaming yet fast in delivery, with a very close recording that almost has no background noise."

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu()

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [15]:
prompt

'Husein zolkepli sangat comel dan kacak suka makan cendol'

In [16]:
audio_arr.numpy().squeeze().shape

(181760,)

In [17]:
import IPython.display as ipd
ipd.Audio(audio_arr.numpy().squeeze(), rate = 44100)

In [20]:
# from huggingface_hub import login

# login()

In [36]:
model.push_to_hub('mesolitica/malay-parler-tts-mini-v1')

model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/malay-parler-tts-mini-v1/commit/7876d8b6e616106c7da347ec3fad8b2891589b59', commit_message='Upload ParlerTTSForConditionalGeneration', commit_description='', oid='7876d8b6e616106c7da347ec3fad8b2891589b59', pr_url=None, pr_revision=None, pr_num=None)

In [37]:
tokenizer.push_to_hub('mesolitica/malay-parler-tts-mini-v1')

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/mesolitica/malay-parler-tts-mini-v1/commit/7876d8b6e616106c7da347ec3fad8b2891589b59', commit_message='Upload tokenizer', commit_description='', oid='7876d8b6e616106c7da347ec3fad8b2891589b59', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
import soundfile as sf

speakers = [
    'Yasmin',
    'Osman',
    'Bunga',
    'Ariff',
    'Ayu',
    'Kamarul',
    'Danial',
    'Elina',
]


for s in speakers:

    description = f"{s}'s voice, with a very close recording that almost has no background noise."

    input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
    prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
    audio_arr = generation.cpu()
    sf.write(f'{s}.mp3', audio_arr.numpy().squeeze(), 44100)